# Assignment 3: Language Modelling with LSTM networks

In this assignment, you will implement an LSTM based language model. We strongly recommend to finish first _lab 4_, which is closely related and is much simpler.

## Setup

First, let's load the data as before.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
import random

# Let's do 2-way positive/negative classification instead of 5-way
easy_label_map = {0:0, 1:0, 2:None, 3:1, 4:1}

def load_sst_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            
            # Strip out the parse information and the phrase labels---we don't need those here
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)

    random.seed(1)
    random.shuffle(data)
    return data

sst_home = 'drive/My Drive/Colab Notebooks/dl4nlp_labs/data/trees/'
training_set = load_sst_data(sst_home + '/train.txt')
dev_set = load_sst_data(sst_home + '/dev.txt')
test_set = load_sst_data(sst_home + '/test.txt')

# Note: Unlike with k-nearest neighbors, evaluation here should be fast, and we don't need to
# trim down the dev and test sets. 

Next, we'll convert the data to index vectors.

To simplify your implementation, we'll use a fixed unrolling length of 20. This means that we'll have to expand each sentence into a sequence of 21 word indices. In the conversion process, we'll mark the start of each sentence with a special word symbol `<S>`, mark the end of each sentence (if it occurs within the first 21 words) with a special word symbol `</S>`, mark extra tokens after `</S>` with a special word symbol `<PAD>`, and mark out-of-vocabulary words with `<UNK>`, for unknown. As in the previous assignment, we'll use a very small vocabulary for this assignment, so you'll see `<UNK>` often.

In [ ]:
import collections
import numpy as np

def sentence_to_padded_index_sequence(datasets):
    '''Annotates datasets with feature vectors.'''
    
    START = "<S>"
    END = "</S>"
    END_PADDING = "<PAD>"
    UNKNOWN = "<UNK>"
    SEQ_LEN = 21
    
    # Extract vocabulary
    def tokenize(string):
        return string.lower().split()
    
    word_counter = collections.Counter()
    for example in datasets[0]:
        word_counter.update(tokenize(example['text']))
    
    vocabulary = set([word for word in word_counter if word_counter[word] > 25])
    vocabulary = list(vocabulary)
    vocabulary = [START, END, END_PADDING, UNKNOWN] + vocabulary
        
    word_indices = dict(zip(vocabulary, range(len(vocabulary))))
    indices_to_words = {v: k for k, v in word_indices.items()}
        
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['index_sequence'] = np.zeros((SEQ_LEN), dtype=np.int32)
            
            token_sequence = [START] + tokenize(example['text']) + [END]
            
            for i in range(SEQ_LEN):
                if i < len(token_sequence):
                    if token_sequence[i] in word_indices:
                        index = word_indices[token_sequence[i]]
                    else:
                        index = word_indices[UNKNOWN]
                else:
                    index = word_indices[END_PADDING]
                example['index_sequence'][i] = index
    return indices_to_words, word_indices
    
indices_to_words, word_indices = sentence_to_padded_index_sequence([training_set, dev_set, test_set])

In [ ]:
print(training_set[18])
print(len(word_indices))

## Assignments: 
### Part 1: Implementation

Now, using the starter code and hyperparameter values provided below, implement an LSTM language model with dropout on the non-recurrent connections. Use the standard form of the LSTM reflected in the slides (without peepholes). **You should only have to edit the marked sections of code to build the base LSTM**, though implementing dropout properly may require small changes to the main training loop and to brittle_sampler().

**Don't use any TensorFlow code that is specifically built for RNNs**. If a TF function has 'recurrent', 'sequence', 'LSTM', or 'RNN' in its name, you should built it yourself instead of using it. (Your version will likely be much simpler, by the way, since these built in methods are powerful but fairly complex and potentially confusing.)

We won't be evaluating our model in the conventional way (perplexity on a held-out test set) for a few reasons: to save time, because we have no baseline to compare against, and because overfitting the training set is a less immediate concern with these models than it was with sentence classifiers. Instead, we'll use the value of the cost function to make sure that the model is converging as expected, and we'll use samples drawn from the model to qualitatively evaluate it.

**Tips**: 
- Check the code for the GRU based sentiment classifier (lab 4), specially the part where the RNN structure is defined.
- You'll need to use `tf.nn.embedding_lookup()`, `tf.nn.sparse_softmax_cross_entropy_with_logits()`, and `tf.split()` at least once each. All three should be easy to Google, though the last homework and the last exercise should show examples of the first two.
- As before, you'll want to initialize your trained parameters using something like `tf.random_normal(..., stddev=0.1)`

**TODOS:**
- **TODO1**: Define the parameters of the LSTM (check the given slides in class)
- **TODO2**: Build the LSTM LM (follow the instructions in the code-comments)


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
tf.__version__

In [ ]:
class LanguageModel:
    def __init__(self, vocab_size, sequence_length):
        # Define the hyperparameters
        self.learning_rate = 0.3  # Should be about right
        self.training_epochs = 250  # How long to train for - chosen to fit within class time
        self.display_epoch_freq = 1  # How often to test and print out statistics
        self.dim = 32  # The dimension of the hidden state of the RNN
        self.embedding_dim = 16  # The dimension of the learned word embeddings
        self.batch_size = 256  # Somewhat arbitrary - can be tuned, but often tune for speed, not accuracy
        self.vocab_size = vocab_size  # Defined by the file reader above
        self.sequence_length = sequence_length  # Defined by the file reader above
        self.rate = 0.25  # Used in dropout (at training time only, not at sampling time)
        
        #### Start main editable code block ####
        self.trainable_variables = []
        # Your model should populate the following four python lists.
        # self.logits should contain one [batch_size, vocab_size]-shaped TF tensor of logits 
        #   for each of the 20 steps of the model.
        # self.costs should contain one [batch_size]-shaped TF tensor of cross-entropy loss 
        #   values for each of the 20 steps of the model.
        # self.h and c should each start contain one [batch_size, dim]-shaped TF tensor of LSTM
        #   activations for each of the 21 *states* of the model -- one tensor of zeros for the 
        #   starting state followed by one tensor each for the remaining 20 steps.
        # Don't rename any of these variables or change their purpose -- they'll be needed by the
        # pre-built sampler.
        
        self.h_zero = tf.zeros([self.batch_size, self.dim])
        self.c_zero = tf.zeros([self.batch_size, self.dim])
        
    
    def model(self,x,rate,sample=False,h_zero=None,c_zero=None):
        def step(x, next_x, h_prev, c_prev):
            # TODO
            return logits, costs, h_prev, c_prev

        self.x_slices = tf.split(x, self.sequence_length, 1)
        all_logits = []
        all_costs = []
        
        if h_zero != None and c_zero != None:
          self.h = [h_zero]
          self.c = [c_zero]
        else:
          self.h = [self.h_zero]
          self.c = [self.c_zero]
        
        x = tf.reshape(self.x_slices[0], [-1])
        
        #TODO unroll
        
        return all_logits,all_costs

        #### End main editable code block ####
        

    def train(self, training_data):
        def get_minibatch(dataset, start_index, end_index):
            indices = range(start_index, end_index)
            vectors = np.vstack([dataset[i]['index_sequence'] for i in indices])
            return vectors
        
       
        print('Training.')

        # Training cycle
        for epoch in range(self.training_epochs):
            random.shuffle(training_set)
            avg_cost = 0.
            total_batch = int(len(training_set) / self.batch_size)
            
            # Loop over all batches in epoch
            for i in range(total_batch):
                # Assemble a minibatch of the next B examples
                minibatch_vectors = np.int32(get_minibatch(training_set, self.batch_size * i, self.batch_size * (i + 1)))

                # Run the optimizer to take a gradient step, and also fetch the value of the 
                # cost function for logging
                with tf.GradientTape() as tape:
                  logits,costs = self.model(minibatch_vectors,self.rate)
                  # Sum costs for each word in each example, but average cost across examples.
                  costs_tensor = tf.concat([tf.expand_dims(cost, 1) for cost in costs], 1)
                  cost_per_example = tf.reduce_sum(costs_tensor, 1)
                  total_cost = tf.reduce_mean(cost_per_example)
            
                # This performs the main SGD update equation
                gradients = tape.gradient(total_cost, self.trainable_variables)
                optimizer = tf.optimizers.SGD(self.learning_rate)
                optimizer.apply_gradients(zip(gradients, self.trainable_variables))
                                                            
                # Compute average loss
                avg_cost += total_cost / (total_batch * self.batch_size)
                
            # Display some statistics about the step
            if (epoch+1) % self.display_epoch_freq == 0:
                tf.print("Epoch:", (epoch+1), "Cost:", avg_cost, "Sample:", self.sample())
    
    def sample(self):
        # This samples a sequence of tokens from the model starting with <S>.
        # We only ever run the first timestep of the model, and use an effective batch size of one
        # but we leave the model unrolled for multiple steps, and use the full batch size to simplify 
        # the training code. This slows things down.

        def brittle_sampler():
            # The main sampling code. Can fail randomly due to rounding errors that yield probibilities
            # that don't sum to one.
            
            word_indices = [0] # 0 here is the "<S>" symbol
            for i in range(self.sequence_length - 1):
                dummy_x = np.zeros((self.batch_size, self.sequence_length),dtype=np.int32)
                dummy_x[0][0] = word_indices[-1]
                model_h = None
                model_c = None
                if i > 0:
                    model_h = h
                    model_c = c
                h, c, logits = self.model(dummy_x,0.0,sample=True,h_zero=model_h,c_zero=model_c)
                logits = logits[0, :] # Discard all but first batch entry
                exp_logits = np.exp(logits - np.max(logits))
                distribution = exp_logits / exp_logits.sum()
                sampled_index = np.flatnonzero(np.random.multinomial(1, distribution))[0]
                word_indices.append(sampled_index)
            words = [indices_to_words[index] for index in word_indices]
            return ' '.join(words)
        
        while True:
            try:
              sample = brittle_sampler()
              return sample
            except ValueError as e:  # Retry if we experience a random failure.
              pass

Now let's train it.

Once you're confident your model is doing what you want, let it run for the full 250 epochs. This will take some time—likely between five and thirty minutes. If it much longer on a reasonably modern laptop—more than an hour—that suggests serious problems with your implementation. A properly implemented model with dropout should reach an average cost of less than 0.22 quickly, and then slowly improve from there. We train the model for a fairly long time because these small improvements in cost correspond to fairly large improvements in sample quality.

Samples from a trained models should have coherent portions, but they will not resemble interpretable English sentences. Here are three examples from a model with a cost value of 0.202:

`<S> the good <UNK> and <UNK> and <UNK> <UNK> with predictable and <UNK> , but also does one of -lrb- <UNK>`

`<S> <UNK> has <UNK> actors seems done <UNK> would these <UNK> <UNK> to <UNK> <UNK> <UNK> 're <UNK> to mind .`

`<S> an action story that was because the <UNK> <UNK> are when <UNK> as ``` <UNK> '' ' it is any`

`-lrb-` and `-rrb` are the way that left and right parentheses are represented in the corpus.

In [ ]:
model = LanguageModel(len(word_indices), 21)
model.train(training_set)

Now we can draw as many samples as we like.

In [ ]:
model.sample()

### Part 2: Questions

**Question 1:** Looking at the samples that your model produced towards the end of training, point out three properties of (written) English that it seems to have learned.

**Question 2:** If we could make the model as big as we wanted, train as long as we wanted, and adjust or remove dropout at will, could we ever get the model to reach a cost value of 0.0? In a single sentence, say why.

**Question 3:** Give an example of a situation where the LSTM language model's ability to propagate information across many steps (when trained for long enough, at least) would cause it to reach a better cost value than a model like a simple RNN without that ability. (Answer in one sentence or so.)

**Question 4:** Would the model be any worse if we were to just delete unknown words instead of using an `<UNK>` token? (Answer in one sentence or so.)

# Atribution:
Adapted by Oier Lopez de Lacalle, Olatz Perez de Viñaspre and Ander Barrena, based on a notebook by Sam Bowman at NYU